In [113]:
import pandas as pd
import numpy as np
import joblib

#load model
loaded_model = joblib.load('linear_regression_model.pkl')

#read data
df_final = pd.read_csv('df_final.csv')
df_final.set_index('date', inplace=True)

#web_visitor
df_visitor_forecast = pd.read_csv('webvisitor_forecast_2.csv')
df_visitor_forecast = df_visitor_forecast[['forecasted_value']]

# Define the forecast horizon as a variable
forecast_horizon = 73
start_date = pd.to_datetime(df_final.index[-1]) + pd.DateOffset(days=1)
date_range = pd.date_range(start_date, periods=forecast_horizon, freq='D')
forecast_df = pd.DataFrame(index=date_range, columns=["forecasted_value"])

end_time = pd.to_datetime(start_time) + pd.DateOffset(days=forecast_horizon - 1)

# Generate the time index
time_index = pd.date_range(start=start_date, end=end_time, freq='D')

# Create a sample DataFrame with this time index
data = {'onboarded_shift_1_week': [7] * len(time_index)}  # Creating a column with constant values for example
df_add = pd.DataFrame(data, index=time_index)

#lawyer online
df_add['lawyer_online_count'] = df_add.index.weekday < 5 
weekday_value = 10
weekend_value = 3
df_add['lawyer_online_count'] = df_add['lawyer_online_count'].apply(lambda x: weekday_value if x else weekend_value)

forecast_array = []  # Renamed from 'forecast_value'
latest_data_row = df_final.iloc[-1, 0:7].to_numpy().reshape(1, -1)

for i in range(forecast_horizon):
    latest_data_row_1 = np.append(latest_data_row, df_visitor_forecast.iloc[i])
    latest_data_row_2 = np.append(latest_data_row_1, df_add.iloc[i][0])
    latest_data_row_3 = np.append(latest_data_row_2, df_add.iloc[i][1])
    latest_data_row_4 = latest_data_row_3.reshape(1, -1)
    predicted_result = loaded_model.predict(latest_data_row_4)
    
    new_data_row = np.concatenate((predicted_result, latest_data_row_4[0]))
#     print(new_data_row)
    
    forecast_array.append(new_data_row)
    
    latest_data_row = new_data_row[:-4].reshape(1, -1)
    
forecast_value = [array[0] for array in forecast_array]
forecast_df["forecasted_value"] = forecast_value

/Users/fadilrisdian/miniconda3/envs/perqara-zth/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/fadilrisdian/miniconda3/envs/perqara-zth/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/fadilrisdian/miniconda3/envs/perqara-zth/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/fadilrisdian/miniconda3/envs/perqara-zth/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/fadilrisdian/miniconda3/envs/perqara-zth/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid fea

In [114]:
forecast_df

,forecasted_value
2023-10-20,20.210559
2023-10-21,10.490220
2023-10-22,11.802685
2023-10-23,17.391462
2023-10-24,12.369034
...,...
2023-12-27,20.728056
2023-12-28,22.967415
2023-12-29,20.495689
2023-12-30,12.317102
